In [26]:
import pandas as pd
import numpy as np
import sklearn
import torch
from sklearn.preprocessing import StandardScaler

In [59]:
class TitanicData:
    def __init__(self, train_path='./train.csv', test_path='./test.csv'):
        self.df_train = pd.read_csv(train_path, encoding='utf8')
        self.df_test = pd.read_csv(test_path, encoding='utf8')
        self._init_train()
        self._init_test()
        self.all_x = pd.concat([self.df_train.iloc[:,:-2], self.df_test], axis=0)
        # _scaler = StandardScaler()
        # np_all_x = _scaler.fit_transform(self.all_x)
        # self.all_x = pd.DataFrame(data=np_all_x, columns=self.all_x.columns)

    def _init_train(self):
        self.df_train.sort_values(by=['Ticket'], ascending=[True], inplace=True)
        self.df_train.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
        self.df_train['Age'].interpolate(inplace=True)
        self.df_train['Fare'].interpolate(inplace=True)
        self.df_train['Embarked'].fillna(value='S', inplace=True)
        self.df_train.sort_values(by=['PassengerId'], ascending=[True], inplace=True)
        self.df_train = pd.get_dummies(data=self.df_train, columns=['Sex', 'Pclass', 'Embarked', 'Survived'])

    def _init_test(self):
        self.df_test.sort_values(by=['Ticket'], ascending=[True], inplace=True)
        self.df_test.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
        self.df_test['Age'].interpolate(inplace=True)
        self.df_test['Fare'].interpolate(inplace=True)
        self.df_test['Embarked'].fillna(value='S', inplace=True)
        self.df_test.sort_values(by=['PassengerId'], ascending=[True], inplace=True)
        self.df_test = pd.get_dummies(data=self.df_test, columns=['Sex', 'Pclass', 'Embarked'])


    def get_x(self, idx: int):
        pass


titanic = TitanicData()
titanic.df_train.iloc[:,:-2]

,PassengerId,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,1,22.000000,1,0,7.2500,0,1,0,0,1,0,0,1
1,2,38.000000,1,0,71.2833,1,0,1,0,0,1,0,0
2,3,26.000000,0,0,7.9250,1,0,0,0,1,0,0,1
3,4,35.000000,1,0,53.1000,1,0,1,0,0,0,0,1
4,5,35.000000,0,0,8.0500,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,27.000000,0,0,13.0000,0,1,0,1,0,0,0,1
887,888,19.000000,0,0,30.0000,1,0,1,0,0,0,0,1
888,889,15.666667,1,2,23.4500,1,0,0,0,1,0,0,1
889,890,26.000000,0,0,30.0000,0,1,1,0,0,1,0,0


In [60]:
all_x = titanic.all_x
all_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Age          1309 non-null   float64
 2   SibSp        1309 non-null   int64  
 3   Parch        1309 non-null   int64  
 4   Fare         1309 non-null   float64
 5   Sex_female   1309 non-null   uint8  
 6   Sex_male     1309 non-null   uint8  
 7   Pclass_1     1309 non-null   uint8  
 8   Pclass_2     1309 non-null   uint8  
 9   Pclass_3     1309 non-null   uint8  
 10  Embarked_C   1309 non-null   uint8  
 11  Embarked_Q   1309 non-null   uint8  
 12  Embarked_S   1309 non-null   uint8  
dtypes: float64(2), int64(3), uint8(8)
memory usage: 71.6 KB


In [54]:
scaler = StandardScaler()
std_x = scaler.fit_transform(all_x.values[:,1:])

In [55]:
std_x

array([[-0.55621546,  0.48128777, -0.4449995 , ..., -0.50976981,
        -0.32204029,  0.65501092],
       [ 0.60814467,  0.48128777, -0.4449995 , ...,  1.96166973,
        -0.32204029, -1.52669211],
       [-0.26512542, -0.47908676, -0.4449995 , ..., -0.50976981,
        -0.32204029,  0.65501092],
       ...,
       [ 0.64453093, -0.47908676, -0.4449995 , ..., -0.50976981,
        -0.32204029,  0.65501092],
       [ 0.60814467, -0.47908676, -0.4449995 , ..., -0.50976981,
        -0.32204029,  0.65501092],
       [-0.24433328,  0.48128777,  0.71076309, ...,  1.96166973,
        -0.32204029, -1.52669211]])

In [56]:
titanic.df_train.describe()

,PassengerId,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Survived_0,Survived_1
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,29.544433,0.523008,0.381594,32.204208,0.352413,0.647587,0.242424,0.206510,0.551066,0.188552,0.086420,0.725028,0.616162,0.383838
std,257.353842,13.869483,1.102743,0.806057,49.693429,0.477990,0.477990,0.428790,0.405028,0.497665,0.391372,0.281141,0.446751,0.486592,0.486592
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,20.750000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,28.000000,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,668.500000,38.000000,1.000000,0.000000,31.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,891.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
